<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Pydantic/pydanticai_weather_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%pip install -qU pydantic-ai nest-asyncio

In [13]:
import os
from google.colab import userdata
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [14]:
# to check if the key is present
print(f"API key present: {'GEMINI_API_KEY' in os.environ}")

API key present: True


In [15]:
# from pydantic_ai.models.groq import GrogModel
# from pydantic_ai.models.openai import OpenAIModel
# from pydantic_ai.models.ollama import OllamaModel
from pydantic_ai.models.gemini import GeminiModel

# OPENAI_MODEL = OpenAIModel("gpt-4o-mini")
# # OLLAMA_MODEL = OllamaModel("llama3.2:3b")
# # GROC_MODEL = GrogModel("llama3-groq-8b-8192-tool-use-preview")

# GROC_MODEL = GrogModel("llama3-groq-8b-8192-tool-use-preview")
GEMINI_MODEL = GeminiModel("gemini-2.0-flash-exp")

# GEMINI_MODEL = GeminiModel("gemini-2.0-flash-thinking-exp-1219")


In [18]:
""" Pydantic AI Weather Agent Example """
import nest_asyncio
nest_asyncio.apply()

from typing import Any

from pydantic import BaseModel, Field

from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.models.gemini import GeminiModel

class Deps(BaseModel):
    """ Default Dependencies """
    weather_api_key: str | None = Field(title='Weather API Key', description='weather service API key')
    geo_api_key: str | None = Field(title='Geo API Key', description='geo service API key')

weather_agent = Agent(
    name='Weather Agent',
    model=GEMINI_MODEL,
    system_prompt=(
        "Be concise, reply with one sentence."
        "Use the 'get_lat_lng' tool to get the latitude and longitude of the locations, "
        "then use the 'get_weather' tool to get the weather."
    ),
    deps_type=Deps,
    # result_type=<response object>,
)

@weather_agent.tool
def get_lat_lng(
    ctx: RunContext[Deps], Location_description: str
) -> dict[str, float]:
    """Get the latitude and longitude of a location.

    Args:
        ctx: The context.
        location_description: A description of a location.
    """

    params = {
        'q': Location_description,
        'api_key': ctx.deps.geo_api_key,
    }

    print('params passed by agent:', params)
    # Simulate an API call to get the latitude and longitude
    if 'London' in Location_description:
        return {"lat": 10.795320, "lng": -55.393958}
    elif 'San Francisco' in Location_description:
        return {"lat": 37.7749, "lng": -122.4194}
    else:
        raise ModelRetry('Could not find the location')

@weather_agent.tool
def get_weather(ctx: RunContext[Deps], lat: float, lng: float) -> dict[str, Any]:
    """Get the weather at a location.

    Args:
        ctx: The context.
        lat: The latitude of the location.
        lng: The longitude of the location.
    """

    if lat == 0 and lng == 0:
        raise ModelRetry("Could not find the location")

    params = {
        "lat": lat,
        "lng": lng,
        "api_key": ctx.deps.weather_api_key,
    }

    # Simulate an API call to get the weather info
    if lat == 10.79532 and lng == -55.393958:
        return {"temp": 70, "description": "Snowing"}
    elif lat == 37.7749 and lng == -122.4194:
        return {"temp": 100, "description": "Windy"}


if __name__ == "__main__":
    deps = Deps(weather_api_key="<weather_api_key>", geo_api_key="<geo_api_key>")

    result = weather_agent.run_sync(
        "What is the weather like in London and in San Francisco, CA?",
        deps=deps
    )

    print("---")
    print("Result:")
    print(result.data)

params passed by agent: {'q': 'London', 'api_key': '<geo_api_key>'}
params passed by agent: {'q': 'San Francisco, CA', 'api_key': '<geo_api_key>'}
---
Result:
The weather in London is Snowing with a temperature of 70, and the weather in San Francisco is Windy with a temperature of 100.



In [ ]:
%python -m pydantic_ai_examples.question_graph